In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt

torch.cuda.set_device(0)

In [2]:
#from multiprocessing import Pool as ThreadPool
import os 
import numpy as np


#pool = ThreadPool(12) 
root_dir = '/disk1/0721_fullfill/'
npyList = os.listdir(root_dir)
npyList = [root_dir+n for n in npyList]
result = []
for i in npyList:
    result.append(np.load(i))
#result = pool.map(np.load, npyList)
result = np.concatenate(result)


from imgaug import augmenters as iaa

result_list = [result] 
seq = iaa.Sequential([
    iaa.Flipud(1), 
    #iaa.ContrastNormalization((1.40,1.60)),
])
seq2= iaa.Sequential([
    iaa.Fliplr(1), 
    #iaa.ContrastNormalization((1.40,1.60)),
])
seq3 = iaa.Sequential([
    iaa.Flipud(1), 
    iaa.Fliplr(1), 
    #iaa.ContrastNormalization((1.40,1.60)),
])

result_list.append(seq.augment_images(result))
result_list.append(seq2.augment_images(result))
result_list.append(seq3.augment_images(result))
result = np.concatenate(result_list,axis=0)
print(result.shape)

X = np.asarray([x.transpose((2,0,1)) for x in result])
X = X.astype(np.float32)/(255.0/2) - 1.0

(19876, 32, 32, 3)


In [3]:
import cv2
import numpy as np
batchsize = 10

#X_train = bonemarrow_cell()
#X_train = functions.bonemarrow_cell()
X_train = X
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
train_loader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize, num_workers=4)

dataiter = iter(train_loader)

In [4]:
rand=128
conti = 128

dis=0
dis_category = 0

class avgpool(nn.Module):
    def __init__(self, up_size=0):
        super(avgpool, self).__init__()
        
    def forward(self, x):
        out_man = (x[:,:,::2,::2] + x[:,:,1::2,::2] + x[:,:,::2,1::2] + x[:,:,1::2,1::2]) / 4
        return out_man
    
class ResidualBlock(nn.Module):

    def __init__(self, in_dim, out_dim, resample=None, up_size=0):
        super(ResidualBlock, self).__init__()
        
        if resample == 'up':
            self.bn1 = nn.BatchNorm2d(in_dim)
            self.conv1 = nn.Conv2d(in_dim, out_dim, 3, 1, 1, bias=True)
            self.upsample = torch.nn.Upsample(up_size,2)
            self.upsample_conv = nn.Conv2d(in_dim, out_dim, 1, 1, 0, bias=True)
            self.conv2 = nn.Conv2d(out_dim, out_dim, 3, 1, 1, bias=True)
            self.bn2 = nn.BatchNorm2d(out_dim)
            
        elif resample == 'down':
            self.conv1 = nn.Conv2d(in_dim, out_dim, 3, 1, 1, bias=True)
            self.conv2 = nn.Conv2d(out_dim, out_dim, 3, 1, 1, bias=True)
            self.pool = avgpool()
            self.pool_conv = nn.Conv2d(in_dim, out_dim, 1, 1, 0, bias=True)
        
        elif resample == None:
            self.conv1 = nn.Conv2d(in_dim, out_dim, 3, 1, 1, bias=True)
            self.conv2 = nn.Conv2d(out_dim, out_dim, 3, 1, 1, bias=True)
            
        self.resample = resample

    def forward(self, x):
        
        if self.resample == None:
            shortcut = x
            output = x
            
            output = nn.functional.relu(output)
            output = self.conv1(output)
            output = nn.functional.relu(output)
            output = self.conv2(output)
            
        elif self.resample == 'up':
            shortcut = x
            output = x
            
            shortcut = self.upsample(shortcut) #upsampleconv
            shortcut = self.upsample_conv(shortcut)
            
            output = self.bn1(output)
            output = nn.functional.relu(output)
            output = self.conv1(output)

            output = self.bn2(output)
            output = nn.functional.relu(output)
            output = self.upsample(output) #upsampleconv
            output = self.conv2(output)
                        
        elif self.resample == 'down':
            shortcut = x
            output = x
            
            shortcut = self.pool_conv(shortcut) #convmeanpool
            shortcut = self.pool(shortcut)
            
            output = nn.functional.relu(output)
            output = self.conv1(output)
            
            output = nn.functional.relu(output)
            output = self.conv2(output)    #convmeanpool
            output = self.pool(output)
            
        return output+shortcut

class ResidualBlock_thefirstone(nn.Module):

    def __init__(self, in_dim, out_dim, resample=None, up_size=0):
        super(ResidualBlock_thefirstone, self).__init__()
        
        self.conv1 = nn.Conv2d(in_dim, out_dim, 3, 1, 1, bias=True)
        self.conv2 = nn.Conv2d(out_dim, out_dim, 3, 1, 1, bias=True)
        self.pool = avgpool()
        self.pool_conv = nn.Conv2d(in_dim, out_dim, 1, 1, 0, bias=True)
        
    def forward(self, x):
        
        shortcut = x
        output = x
        
        shortcut = self.pool(shortcut) #meanpoolconv
        shortcut = self.pool_conv(shortcut)

        output = self.conv1(output)
        output = nn.functional.relu(output)
        output = self.conv2(output) #convmeanpool
        output = self.pool(output)
            
        return output+shortcut

class generator(nn.Module):

    def __init__(self, rand=128):
        super(generator, self).__init__()
        self.rand = rand
        self.linear = nn.Linear(rand  ,2048, bias=True)
        self.layer_up_1 = ResidualBlock(128, 128, 'up', up_size=8)
        self.layer_up_2 = ResidualBlock(128, 128, 'up', up_size=16)
        self.layer_up_3 = ResidualBlock(128, 128, 'up', up_size=32)
        self.bn1 = nn.BatchNorm2d(128)
        self.conv_last = nn.Conv2d(128, 3, 3, 1, 1, bias=True)

    def forward(self, x):
        x = x.view(-1,self.rand)
        x = self.linear(x)
        x = x.view(-1,128,4,4)
        x = self.layer_up_1(x)
        x = self.layer_up_2(x)
        x = self.layer_up_3(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.conv_last(x)
        x = nn.functional.tanh(x)
        return x
    
netG = generator(rand = rand+dis*dis_category+conti)

class discriminator(nn.Module):

    def __init__(self):
        super(discriminator, self).__init__()
        self.layer_down_1 = ResidualBlock_thefirstone(3, 128)
        self.layer_down_2 = ResidualBlock(128, 128, 'down')
        self.layer_none_1 = ResidualBlock(128, 128, None)
        self.layer_none_2 = ResidualBlock(128, 128, None)
        #self.mean_pool = nn.AvgPool2d(8,1,0)
        #self.linear = nn.Linear(128,1, bias=True)
        #self.linear2 = nn.Linear(128,10, bias=True)
        
    def forward(self, x):
        x = self.layer_down_1(x)
        x = self.layer_down_2(x)
        x = self.layer_none_1(x)
        x = self.layer_none_2(x)
        #x = self.mean_pool(x)
        x = nn.functional.relu(x)
        x = x.mean(2).mean(2)
        x = x.view(-1, 128)

        #shortcut = x
        #output = x
        
        #output = self.linear(output)
        #shortcut= self.linear2(shortcut)
        
        #return output.view(-1,1,1,1), shortcut.view(-1,10,1,1)
        return x
    
netD = discriminator()

In [5]:
class _netD_D(nn.Module):
    def __init__(self):
        super(_netD_D, self).__init__()
        self.linear = nn.Linear(128,1, bias=True)
        #self.conv = nn.Conv2d(4096, 1, 1, 1, 0, bias=True)
        
    def forward(self, x):
        x = self.linear(x)
        return x.view(-1,1,1,1)

'''
class _netD_Q(nn.Module):
    def __init__(self, nd = 10):
        super(_netD_Q, self).__init__()
        # input is Z, going into a convolution
        #self.conv = nn.Conv2d(4096, 128, 1, 1, 0, bias=True)
        #self.relu = nn.LeakyReLU(0.2, inplace=True)
        #self.conv2 = nn.Conv2d(128, nd, 1, 1, 0, bias=True)
        self.softmax = nn.LogSoftmax()
        #self.linear1 = nn.Linear(4096,128, bias=True)
        #self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.linear2 = nn.Linear(128,nd, bias=True)
        self.nd = nd

    def forward(self, x):
        #x = self.linear1(x)
        #x = self.relu(x)
        x = self.linear2(x)
        x = self.softmax(x)
       # x = x.view(64,10)
        return x.view(-1,self.nd,1,1)
'''

class _netD_Q(nn.Module):
    def __init__(self, nc = 128):
        super(_netD_Q, self).__init__()
        #self.softmax = nn.LogSoftmax()
        self.linear2 = nn.Linear(128,nc, bias=True)
        self.nc = nc

    def forward(self, x):
        #x = self.linear1(x)
        #x = self.relu(x)
        x = self.linear2(x)
        #x = self.softmax(x)
       # x = x.view(64,10)
        return x.view(-1,self.nc,1,1)

netD_D = _netD_D()
#netD_Q = _netD_Q(dis_category)
netD_Q = _netD_Q(conti)

print(netD_D)
print(netD_Q)

_netD_D (
  (linear): Linear (128 -> 1)
)
_netD_Q (
  (linear2): Linear (128 -> 128)
)


In [6]:
def uniform(stdev, size):
    return np.random.uniform(
                low=-stdev * np.sqrt(3),
                high=stdev * np.sqrt(3),
                size=size
            ).astype('float32')

def initialize_conv(m,he_init=True):
    fan_in = m.in_channels * m.kernel_size[0]**2
    fan_out = m.out_channels * m.kernel_size[0]**2 / (m.stride[0]**2)

    #fan_in /= 2.
    #fan_out /= 2.

    if m.kernel_size[0]==3:
        filters_stdev = np.sqrt(4./(fan_in+fan_out))
        #print("3:",m)
    else: # Normalized init (Glorot & Bengio)
        filters_stdev = np.sqrt(2./(fan_in+fan_out))
        #print("1:",m)
        
    filter_values = uniform(
                    filters_stdev,
                    (m.kernel_size[0], m.kernel_size[0], m.in_channels, m.out_channels)
                )
    
    return filter_values

def initialize_linear(m):
    weight_values = uniform(
                np.sqrt(2./(m.in_features+m.out_features)),
                (m.in_features, m.out_features)
            )
    return weight_values

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        weight = torch.from_numpy(initialize_conv(m))
        m.weight.data.copy_(weight,broadcast=False)
        m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        weight_values = torch.from_numpy(initialize_linear(m))
        m.weight.data.copy_(weight_values,broadcast=False)
        m.bias.data.fill_(0)

netG.apply(weights_init)
netD.apply(weights_init)
netD_Q.apply(weights_init)
netD_D.apply(weights_init)

netD, netG, netD_D, netD_Q = netD.cuda(), netG.cuda(), netD_D.cuda(), netD_Q.cuda()

ld = 1e-4
lg = 1e-4
lq_d = 1e-4
lq_g = 1e-4

optimizerD = optim.Adam([
                {'params': netD.parameters()},
                {'params': netD_D.parameters()}
            ], ld, betas=(0.5, 0.9))

optimizerG = optim.Adam([
                {'params': netG.parameters()},
                {'params': netD.parameters()},
                {'params': netD_Q.parameters()}
            ],lg, betas=(0.5, 0.9))
 
optimizerQ_D = optim.Adam([
                {'params': netD.parameters()},
            ], ld, betas=(0.5, 0.9))

optimizerQ_G = optim.Adam([
                {'params': netG.parameters()},            
            ], lg, betas=(0.5, 0.9))

input = torch.FloatTensor(batchsize, 3, 32, 32)
noise = torch.FloatTensor(batchsize, rand+conti,1 ,1 )

fixed_noise = torch.FloatTensor(np.random.multinomial(batchsize, 10*[0.1], size=1))
c = torch.randn(batchsize, 10)
c_conti = torch.FloatTensor(np.random.uniform(-1,1,(batchsize,conti)))
z = torch.randn(batchsize, rand)

label = torch.FloatTensor(1)

real_label = 1
fake_label = 0

criterion = nn.BCELoss()
criterion_logli = nn.NLLLoss(size_average=True)
criterion_mse = nn.MSELoss()

criterion, criterion_logli, criterion_mse = criterion.cuda(), criterion_logli.cuda(), criterion_mse.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
z, c, c_conti = z.cuda(), c.cuda(), c_conti.cuda()

In [7]:
def sample_c(batchsize, dis_category=10):
    rand_c = np.zeros((batchsize,dis_category),dtype='float32')
    for i in range(0,batchsize):
        rand = np.random.multinomial(1, dis_category*[1/float(dis_category)], size=1)
        rand_c[i] = rand

    label_c = np.argmax(rand_c,axis=1)
    label_c = torch.LongTensor(label_c.astype('int'))
    rand_c = torch.from_numpy(rand_c.astype('float32'))
    return rand_c,label_c

def zero_grad():
    netD.zero_grad()
    netD_Q.zero_grad()
    netD_D.zero_grad()
    netG.zero_grad()
        
def fix_noise(dis=1, rand=128, dis_category=10, row=10):
    
    fixed_z = np.random.randn(row, rand).repeat(dis_category,axis=0)
    changing_dis = np.zeros((row*dis_category,dis_category),dtype = np.float32)
    list = [n for n in range(0,dis_category)]*row
    for i in range(0,row*dis_category):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(row*dis_category,rand+dis*dis_category,1,1)

def exp_lr_scheduler(optimizer, iteration, init_lr=0.001, lr_decay_iter=1):
    
    lr = init_lr* (1.0 - iteration / float(lr_decay_iter))
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    return optimizer

def exp_lr_scheduler_2(optimizer, iteration, init_lr=0.001, lr_decay_iter=1):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    if iteration<200000:
        lr = init_lr + init_lr*(1*(iteration // 20000))
    if iteration>=200000:
        lr = init_lr*2 - init_lr*(1*(iteration // 20000))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

one = torch.FloatTensor([1])
mone = one * -1
one = one.cuda()
mone = mone.cuda()
fixed_noise = torch.from_numpy(fix_noise(dis_category=dis_category)).cuda()

In [ ]:
def calc_gradient_penalty(netD_D, netD, real_data, fake_data,lamda,batch_size):
    #print real_data.size()
    alpha = torch.rand(batch_size,1,1,1)
    #print (real_data.size())
    alpha = alpha.expand(real_data.size())
    alpha = alpha.cuda()

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    interpolates = interpolates.cuda()
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD_D(netD(interpolates))#.view(batch_size,-1)

    gradients, = autograd.grad(outputs=disc_interpolates.sum(), inputs=interpolates,
                              create_graph=True)
    
    #gradients, = autograd.grad(outputs=disc_interpolates.sum(), inputs=interpolates,
                              #grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                              #create_graph=True, retain_graph=True, only_inputs=True)
    #gradients*gradients
    
    gradient_penalty = ((gradients.view(batch_size,-1).norm(2, dim=1) - 1) ** 2).mean()* lamda
    return gradient_penalty


In [ ]:
import torch.autograd as autograd
import time

gen_iterations = 0
lamda = 10
cont_lamda = 1
end = time.time()

for epoch in range(100000):

    dataiter = iter(train_loader)
    i = 0
    
    while i < len(train_loader):
        
        for p in netD.parameters(): 
            p.requires_grad = True 
        for p in netD_D.parameters(): 
            p.requires_grad = True 

        for iter_d in range(0,5):
            if i >=len(train_loader):
                continue
                                
            image_, _ = dataiter.next()
            _batchsize = image_.size(0)
            image_ = image_.cuda()
            i +=1
            input.resize_as_(image_).copy_(image_)
            inputv = Variable(input)
            
            #train with real
            errD_real = netD_D(netD(inputv)).mean()
            errD_real.backward(mone)
            
            # train with fake
            #rand_c,label_c = sample_c(_batchsize,dis_category=dis_category)
            #rand_c = rand_c.cuda()
            #c.resize_as_(rand_c).copy_(rand_c)
            c_conti.resize_(_batchsize, conti, 1, 1).uniform_(-1,1)
            z.resize_(_batchsize, rand, 1, 1).normal_(0, 1)
            #noise = torch.cat([c,z],1)
            noise = torch.cat([c_conti,z],1)
            noise_resize = noise.view(_batchsize,rand+conti,1,1)
            noisev = Variable(noise_resize, volatile = True)
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD_D(netD(inputv)).mean()
            errD_fake.backward(one)
            
            # train with gradient penalty
            gradient_penalty = calc_gradient_penalty(netD_D,netD, input, fake.data,lamda,_batchsize)
            gradient_penalty.backward()
            
            
            Q_c_given_x = netD_Q(netD(fake))
            square_loss = criterion_mse(Q_c_given_x, Variable(c_conti,requires_grad=False))
            mi_loss = 1*square_loss
            
            D_cost = -errD_real + errD_fake + gradient_penalty+mi_loss

            optimizerD.step()

            '''
            # update Q
            zero_grad()
            noisev = Variable(noise_resize)
            G_sample = netG(noisev)
            Q_c_given_x = netD_Q(netD(G_sample))
            square_loss = criterion_mse(Q_c_given_x, Variable(c_conti,requires_grad=False))
            #square_loss = square_loss.mean()
            #crossent_loss = criterion_logli(Q_c_given_x ,Variable(label_c.cuda()))
            
            mi_loss = 1*square_loss
            mi_loss.backward()
            
            optimizerQ_D.step()
            optimizerQ_G.step()
            '''
    # update G  
        for p in netD.parameters(): 
            p.requires_grad = False 
        for p in netD_D.parameters(): 
            p.requires_grad = False

        zero_grad()
        c_conti.resize_(_batchsize, conti, 1, 1).uniform_(-1,1)
        z.resize_(_batchsize, rand, 1, 1).normal_(0, 1)
            #noise = torch.cat([c,z],1)
        noise = torch.cat([c_conti,z],1)
        noise_resize = noise.view(_batchsize,rand+conti,1,1)
        noisev = Variable(noise_resize)
        fake = netG(noisev)
        errG = netD_D(netD(fake)).mean()
        errG.backward(mone,retain_variables=True)
        
        for p in netD.parameters(): 
            p.requires_grad = True 
        for p in netD_D.parameters(): 
            p.requires_grad = True        
            
        Q_c_given_x = netD_Q(netD(fake)).view(_batchsize, conti)
        square_loss = criterion_mse(Q_c_given_x, Variable(c_conti,requires_grad=False))
        mi_loss = 1*square_loss
        mi_loss.backward()
        
        optimizerG.step()
        gen_iterations += 1
        
        
        # update Q
        '''
        for p in netD.parameters(): 
            p.requires_grad = True 
        for p in netD_D.parameters(): 
            p.requires_grad = True 
        
        zero_grad()
        noisev = Variable(noise_resize)
        G_sample = netG(noisev)
        Q_c_given_x = netD_Q(netD(G_sample)).view(_batchsize, conti)
        #square_loss = (Q_c_given_x - Variable(c_conti).view(1,-1,1,1))**2*cont_lamda
        #square_loss = square_loss.mean()
        #crossent_loss = criterion_logli(Q_c_given_x ,Variable(label_c.cuda()))
        
        square_loss = criterion_mse(Q_c_given_x, Variable(c_conti,requires_grad=False))
        mi_loss = 1*square_loss
        mi_loss.backward()
        optimizerQ_D.step()
        optimizerQ_G.step()
        '''
        
        if gen_iterations % 20 == 0 :
            
            batch_time = time.time() - end
            end = time.time()
            
            #errD = D_real - D_fake
            with open("output_cell_white.txt","a") as f:
                f.write('{0} {1} {2} {3}'.format(batch_time, gen_iterations , -D_cost.data[0] , mi_loss.data[0]) + '\n')
            print ('{0} {1} {2} {3}'.format(batch_time, gen_iterations , -D_cost.data[0] , mi_loss.data[0]))
            #G_sample = netG(Variable(fixed_noise))
            #vutils.save_image(G_sample.data, 'fake_cell_white_10.png',nrow=10,normalize=True)
            vutils.save_image(fake.data, 'fake_128.png',normalize=True)

/usr/local/lib/python2.7/dist-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


14.684720993 20 -34.2114219666 40.2236824036
14.0530378819 40 33.0138168335 13.7289552689
14.0737419128 60 15.3261175156 1.88536906242
14.1343419552 80 9.65149307251 4.54908847809
14.1402218342 100 12.6510248184 1.04353165627
14.2323482037 120 11.1052083969 0.912113845348
14.2183270454 140 14.14427948 0.857152581215
14.2140500546 160 11.8891162872 1.00184595585
14.2651050091 180 7.5283613205 0.571705460548
14.4106390476 200 7.38425827026 0.823793232441
14.2670650482 220 6.01981019974 0.40208029747
14.184098959 240 6.36203432083 0.467750072479
14.2642230988 260 5.18885564804 0.616124629974
14.2549130917 280 5.30729103088 0.436119377613
14.1667790413 300 3.80770206451 0.534380018711
14.3389620781 320 3.4978415966 0.529752492905
14.2282030582 340 4.67249536514 0.525169253349
14.3191399574 360 5.10848045349 0.574217915535
14.3653841019 380 4.3252620697 0.562976062298
14.1598119736 400 4.28765726089 0.630503475666
14.2528970242 420 4.81042861938 0.485614448786
14.204447031 440 3.9351079464 

14.1188139915 3480 2.40317201614 0.400020360947
14.3383169174 3500 4.68916797638 0.362593740225
14.194163084 3520 5.69502449036 0.380170673132
14.1920230389 3540 4.16623210907 0.364924669266
14.1206390858 3560 3.85402870178 0.383557081223
14.0902700424 3580 4.59667921066 0.368728160858
14.0944080353 3600 5.29686641693 0.388652414083
14.3440890312 3620 1.20523452759 0.401563793421
14.0896811485 3640 3.98382091522 0.371189415455
14.1342329979 3660 1.32600080967 0.406156539917
14.1779880524 3680 4.5581817627 0.40834531188
14.1649320126 3700 5.10113382339 0.367642879486
14.3461720943 3720 1.37493550777 0.457036346197
14.356981039 3740 2.74655771255 0.405988991261
14.1837420464 3760 5.14043188095 0.393229305744
14.2193300724 3780 -0.425687968731 0.366875112057
14.227077961 3800 2.61721682549 0.404992878437
14.1855831146 3820 3.29877543449 0.390870630741
14.3697218895 3840 3.58883976936 0.382619708776
14.4239828587 3860 5.22522115707 0.37043428421
14.3725531101 3880 4.04754972458 0.381680965

14.173238039 6920 3.17440080643 0.368225991726
14.4742949009 6940 2.28402686119 0.472310215235
14.1252760887 6960 4.34739494324 0.415245711803
14.2358319759 6980 2.91271924973 0.36258149147
14.16943717 7000 3.19623565674 0.437887907028
14.1627850533 7020 3.06750011444 0.392213881016
14.325248003 7040 2.27063298225 0.375962495804
14.3736400604 7060 3.85008978844 0.396343052387
14.6579399109 7080 2.24988293648 0.445597469807
14.0353507996 7100 2.23043560982 0.400437533855
14.3263339996 7120 3.31758260727 0.395610183477
14.3192360401 7140 1.15833604336 0.375348627567
14.208178997 7160 0.320879161358 0.404474914074
14.2441020012 7180 6.07030630112 0.362863063812
14.4000840187 7200 0.330970138311 0.409080207348
14.2463991642 7220 3.01781487465 0.356991440058
14.2228980064 7240 4.97347784042 0.380742669106
14.3105139732 7260 3.92171192169 0.388402760029
14.1127488613 7280 4.80337762833 0.393530130386
14.4100129604 7300 4.8093957901 0.416306823492
14.1670248508 7320 3.22554779053 0.3592543303

In [ ]:
Q_c_given_x

In [ ]:
square_loss = (Q_c_given_x - Variable(c_conti).view(1,-1,1,1))**2*cont_lamda
square_loss.mean()

In [ ]:
criterion_mse(Q_c_given_x, Variable(c_conti,requires_grad=False)).backward()

In [ ]:
z

In [ ]:
def fix_noise(dis=1, rand=128, dis_category=10, row=10):
    
    fixed_z = np.random.randn(row, rand).repeat(dis_category,axis=0)
    changing_dis = np.zeros((row*dis_category,dis_category),dtype = np.float32)
    list = [n for n in range(0,dis_category)]*row
    for i in range(0,row*dis_category):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(row*dis_category,rand+dis*dis_category,1,1)


In [ ]:
def continueous_noise(rand=128):
    fixed_z = np.random.randn(1, rand).repeat(5 ,axis=0)
    changing_c = np.numpy([[-2],[-1],[0],[1],[2]], dtype=np.float32)
    fixed_c = ([0,0,0,0,0], dtype=np.float32)
    list = []
    for i in range(0,128):
        
    #changing_dis = np.zeros((row*dis_category,dis_category),dtype = np.float32)
    continueous_code = np.zeros((640,128),dtype=int)
    for row in range(0, 128):
        for row_code in range(0,5):
            continueous_code[row_code*128+, ] = 
            continueous_code[row*128]
        
    list = [n for n in range(0,dis_category)]*row
    for i in range(0,row*dis_category):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(row*dis_category,rand+dis*dis_category,1,1)
    
    fixed_z = np.random.randn(row, rand).repeat(dis_category,axis=0)
    changing_dis = np.zeros((row*dis_category,dis_category),dtype = np.float32)
    list = [n for n in range(0,dis_category)]*row
    for i in range(0,row*dis_category):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(row*dis_category,rand+dis*dis_category,1,1)